In [187]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics.pairwise import cosine_similarity

In [188]:
# Read column names
temp_df = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module2/movie_reviews/user_reviews.csv', nrows=1)
# Get all column names except the first two
column_names = temp_df.columns[2:]

# Read data
ratings = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module2/movie_reviews/user_reviews.csv',
                      sep=',',
                      usecols=column_names)
print("shape::", ratings.shape)
print(ratings)
print(ratings.iloc[0,0])

shape:: (600, 2000)
     The Net  Happily N'Ever After  Tomorrowland  American Hero  Das Boot  \
0        0.0                   0.0           0.0            0.0       0.0   
1        0.0                   0.0           0.0            0.0       0.0   
2        0.0                   0.0           0.0            0.0       0.0   
3        0.0                   0.0           0.0            0.0       0.0   
4        0.0                   0.0           0.0            0.0       0.0   
..       ...                   ...           ...            ...       ...   
595      5.0                   0.0           0.0            0.0       0.0   
596      0.0                   0.0           0.0            0.0       0.0   
597      0.0                   0.0           0.0            0.0       0.0   
598      0.0                   0.0           0.0            0.0       0.0   
599      0.0                   0.0           0.0            0.0       0.0   

     Final Destination 3  Licence to Kill  The Hundred-

In [189]:
users = ["Vincent", "Edgar", "Addilyn", "Marlee","Javier"]
df = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module2/movie_reviews/user_reviews.csv')
names = df.iloc[:,1]
print(names)

# Find the corresponding row numbers
user_indices = {}
for user in users:
    index = 0
    for name in names:
        if user == name :
            user_indices[user] = index
            break
        index = index+1

print(user_indices)

0      Vincent
1        Edgar
2      Addilyn
3       Marlee
4       Javier
        ...   
595    Mariana
596        Ivy
597      Kevin
598       Nora
599      Sarai
Name: User, Length: 600, dtype: object
{'Vincent': 0, 'Edgar': 1, 'Addilyn': 2, 'Marlee': 3, 'Javier': 4}


In [190]:
# Calculate user similarity
user_similarity = cosine_similarity(ratings)
print(user_similarity)

[[1.         0.01624925 0.02072233 ... 0.         0.         0.        ]
 [0.01624925 1.         0.         ... 0.         0.         0.02619543]
 [0.02072233 0.         1.         ... 0.01951181 0.         0.05512071]
 ...
 [0.         0.         0.01951181 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.02575758]
 [0.         0.02619543 0.05512071 ... 0.         0.02575758 1.        ]]


In [191]:
def recommend_for_user_modified(user_index, user_similarity, df):
    # Similar Users
    similar_users = user_similarity[user_index]

    # Ensure that df and user_similarity have the same number of users
    if df.shape[0] != len(similar_users):
        raise ValueError("Mismatch in the number of users between df and user_similarity matrix")

    # Only consider ratings where the original rating is greater than 0
    rated_mask = df > 0

    # Weighted Ratings: Calculate the weighted sum of ratings
    # Transpose df and multiply each rating by the similarity, then sum them up, ignoring zeros
    weighted_ratings = np.dot((df * rated_mask).T, similar_users)

    # Convert to Series for sorting and filtering
    recommendations = pd.Series(weighted_ratings, index=df.columns).sort_values(ascending=False)

    # Filtering Out Known Ratings (ratings that are not 0)
    known_ratings = df.columns[df.iloc[user_index] > 0]
    recommendations = recommendations.drop(known_ratings)

    return recommendations

for user in user_indices:
    user_index = user_indices[user]
    recommendations = recommend_for_user_modified(user_index, user_similarity, ratings)
    #print(recommendations)
    #print(recommendations.shape)
    print("user name: "+ user )
    print("Top 5 recommendations: ")
    print(recommendations[:5])



user name: Vincent
Top 5 recommendations: 
Perrier's Bounty                2.403400
Panic Room                      1.843701
Now You See Me 2                1.655318
The Bridge on the River Kwai    1.614550
Bran Nue Dae                    1.612613
dtype: float64
user name: Edgar
Top 5 recommendations: 
Moonrise Kingdom          1.690657
Blast from the Past       1.505775
Enemy of the State        1.470219
Silent House              1.431968
The Fault in Our Stars    1.395809
dtype: float64
user name: Addilyn
Top 5 recommendations: 
Now You See Me 2      1.844762
Ice Princess          1.828805
Maid in Manhattan     1.768886
Heavenly Creatures    1.718856
Red Sky               1.639371
dtype: float64
user name: Marlee
Top 5 recommendations: 
Lilyhammer                               2.011629
The Good Thief                           1.729874
Broken Horses                            1.695285
Ice Princess                             1.660488
Street Fighter: The Legend of Chun-Li    1.654991
d